# **Importamos librerias**

In [1]:
from etl import GetData
from feature_engineer import FeatureEngineer

/Users/jonatanlondonotaborda/Documents/Repositorios/MLOps_CML/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# **ETL**

In [2]:
get_data = GetData()

In [5]:
get_data.download_data()

In [3]:
df = get_data.create_dataset()

In [4]:
df.head()

,type,fixed_acidity,volatile_acidity,citric_acid,residual_sugar,chlorides,free_sulfur_dioxide,total_sulfur_dioxide,density,pH,sulphates,alcohol,quality
0,white,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.0010,3.00,0.45,8.8,6
1,white,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.9940,3.30,0.49,9.5,6
2,white,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.9951,3.26,0.44,10.1,6
3,white,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6
4,white,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6


# **Feature Engineering**

In [5]:
feature_engineer = FeatureEngineer(df)

In [6]:
df_engineered = feature_engineer.create_features()

In [7]:
df_engineered

,fixed_acidity,volatile_acidity,citric_acid,residual_sugar,chlorides,free_sulfur_dioxide,total_sulfur_dioxide,density,pH,sulphates,alcohol,quality
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.00100,3.00,0.45,8.8,6
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.99400,3.30,0.49,9.5,6
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.99510,3.26,0.44,10.1,6
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.99560,3.19,0.40,9.9,6
4,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.99560,3.19,0.40,9.9,6
...,...,...,...,...,...,...,...,...,...,...,...,...
4893,6.2,0.21,0.29,1.6,0.039,24.0,92.0,0.99114,3.27,0.50,11.2,6
4894,6.6,0.32,0.36,8.0,0.047,57.0,168.0,0.99490,3.15,0.46,9.6,5
4895,6.5,NaN,0.19,1.2,0.041,30.0,111.0,0.99254,2.99,0.46,9.4,6
4896,5.5,0.29,0.30,1.1,0.022,20.0,110.0,0.98869,3.34,0.38,12.8,7


In [8]:
numeric_features = ['volatile_acidity', 'residual_sugar', 'density', 'alcohol']
categorical_features = []
target_column = 'quality'
 
test_size = 0.25

# **Step 3: Modelo con MLFlow + Optuna**

mlflow server --backend-store-uri sqlite:///wine_quality.db

In [9]:
from sklearn.ensemble import RandomForestClassifier
from train_with_mlflow_optuna import TrainMlflowOptuna
import mlflow

mlflow.set_tracking_uri("http://127.0.0.1:5000")

In [10]:
# Define RandomForest parameter distributions
param_distributions = {
    'n_estimators': ('int', 50, 200),
    'max_depth': ('int', 5, 30),
    'min_samples_split': ('int', 2, 10),
    'min_samples_leaf': ('int', 1, 5),
    'max_features': ('categorical', ['sqrt', 'log2', None])
}

# Create trainer for RandomForest
trainer = TrainMlflowOptuna(
    df=df_engineered,
    numeric_features=numeric_features,
    categorical_features=categorical_features,
    target_column=target_column,
    model_class=RandomForestClassifier,
    test_size=0.3,
    n_trials=30,
    optimization_metric='f1',  # Optimize for F1 score
    param_distributions=param_distributions,
    model_params={'random_state': 42, 'n_jobs': -1},
    mlflow_setup = mlflow
)

# Run optimization
best_pipeline, run_id, study = trainer.train()

[I 2025-10-04 12:19:56,481] A new study created in memory with name: optuna_RandomForestClassifier
[I 2025-10-04 12:19:56,626] Trial 0 finished with value: 0.4975857844314939 and parameters: {'n_estimators': 159, 'max_depth': 6, 'min_samples_split': 3, 'min_samples_leaf': 5, 'max_features': 'log2'}. Best is trial 0 with value: 0.4975857844314939.


Starting Optuna optimization with 30 trials...
Optimizing for: f1
Model type: RandomForestClassifier


2025/10/04 12:19:56 INFO mlflow.tracking.fluent: Experiment with name 'optuna_RandomForestClassifier' does not exist. Creating a new experiment.
[I 2025-10-04 12:19:57,190] Trial 1 finished with value: 0.5610976784458215 and parameters: {'n_estimators': 186, 'max_depth': 11, 'min_samples_split': 7, 'min_samples_leaf': 5, 'max_features': 'sqrt'}. Best is trial 1 with value: 0.5610976784458215.


🏃 View run 0 at: http://127.0.0.1:5000/#/experiments/1/runs/a9695423409d4c289c031c872a32ea4d
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/1
🏃 View run 1 at: http://127.0.0.1:5000/#/experiments/1/runs/78ed9029ddec4180915e811930e6e344
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/1


[I 2025-10-04 12:19:57,428] Trial 2 finished with value: 0.5717134931943791 and parameters: {'n_estimators': 178, 'max_depth': 16, 'min_samples_split': 7, 'min_samples_leaf': 5, 'max_features': None}. Best is trial 2 with value: 0.5717134931943791.
[I 2025-10-04 12:19:57,567] Trial 3 finished with value: 0.5747200307937766 and parameters: {'n_estimators': 102, 'max_depth': 16, 'min_samples_split': 9, 'min_samples_leaf': 5, 'max_features': 'sqrt'}. Best is trial 3 with value: 0.5747200307937766.


...

🏃 View run 26 at: http://127.0.0.1:5000/#/experiments/1/runs/a429e6ffe79c4f3996ea0dcbf4456512
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/1
🏃 View run 27 at: http://127.0.0.1:5000/#/experiments/1/runs/cb0ffbdb889a47af90428aaf05987f72
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/1


[I 2025-10-04 12:20:01,738] Trial 28 finished with value: 0.49461317791768844 and parameters: {'n_estimators': 113, 'max_depth': 5, 'min_samples_split': 3, 'min_samples_leaf': 2, 'max_features': None}. Best is trial 13 with value: 0.6436037827853707.
[I 2025-10-04 12:20:01,937] Trial 29 finished with value: 0.6441271833031672 and parameters: {'n_estimators': 161, 'max_depth': 19, 'min_samples_split': 4, 'min_samples_leaf': 1, 'max_features': 'log2'}. Best is trial 29 with value: 0.6441271833031672.


🏃 View run 28 at: http://127.0.0.1:5000/#/experiments/1/runs/3abda288a12147a695ee34ca46daaa89
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/1
🏃 View run 29 at: http://127.0.0.1:5000/#/experiments/1/runs/755c1d93816244b3877f7c55bf77db14
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/1

Optimization complete!
Best f1: 0.6441
Best parameters: {'n_estimators': 161, 'max_depth': 19, 'min_samples_split': 4, 'min_samples_leaf': 1, 'max_features': 'log2'}


2025/10/04 12:20:02 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/10/04 12:20:04 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.



Best Model MLflow Run ID: 9e6c63303eb64f079bf19aea61b9885e
Tracking URI: http://127.0.0.1:5000
Train Accuracy: 0.9869
Test Accuracy: 0.6531
🏃 View run best_model_RandomForestClassifier at: http://127.0.0.1:5000/#/experiments/1/runs/9e6c63303eb64f079bf19aea61b9885e
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/1
